In [1]:
from io import StringIO

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdocument import PDFDocument
from pdfminer.converter import TextConverter
from pdfminer.pdfparser import PDFParser
from pdfminer.layout import LAParams, LTLine, LTRect, LTTextBoxHorizontal
from pdfminer.pdfpage import PDFPage

from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfdevice import PDFDevice

from urllib.request import urlretrieve
from difflib import get_close_matches, SequenceMatcher
from validate_docbr import CPF
from os import path, listdir
from pandas import read_csv
# from xlwings import Book
import requests
import nltk

from ast import literal_eval
import json

from dateutil.relativedelta import relativedelta
from datetime import datetime
from json import dumps
import zipfile
import re

import numpy as np
import cv2

In [2]:
today = datetime.now()

# dateUrl = datetime.now()

# while True:
#     try:
#         fileHandle, _ = urlretrieve('http://www.portaltransparencia.gov.br/download-de-dados/servidores/' + dateUrl.strftime('%Y%m') + '_Servidores.zip')
#         break
#     except:
#         dateUrl -= relativedelta(months=1)
#         print('http://www.portaltransparencia.gov.br/download-de-dados/servidores/' + dateUrl.strftime('%Y%m') + '_Servidores.zip')

# zipFile = zipfile.ZipFile(fileHandle, 'r')

# dfsTransparencia = {}
# for fileName in zipFile.namelist():
#     if fileName.split('_')[1] in ['Remuneracao.csv', 'Cadastros.csv']:
#         csvData = zipFile.open(fileName)

#         dfsTransparencia[fileName.split('_')[1]] = read_csv(csvData, sep=';', engine='c', low_memory=False, encoding='latin-1')

# del zipFile, fileHandle, dateUrl, _

In [ ]:
#     def check_portal_transparencia(self):
#         cpfPiecesList = re.findall('\d\d\d?', self.cpf)
#         cpfToFind = f'***.{cpfPiecesList[1]}.{cpfPiecesList[2]}-**'
        
#         remuneracao = dfsTransparencia['Remuneracao.csv'][(dfsTransparencia['Remuneracao.csv']['CPF'] == cpfToFind) & (dfsTransparencia['Remuneracao.csv']['NOME'] == self.name)]
#         cadastro = dfsTransparencia['Cadastros.csv'][(dfsTransparencia['Remuneracao.csv']['CPF'] == cpfToFind) & (dfsTransparencia['Remuneracao.csv']['NOME'] == self.name)]
        
#         assert remuneracao.shape[0] or cadastro.shape[0], f'SERVIDOR NOT FOUND IN "Portal Transparencia" ' + today.strftime('%d/%m/%Y, %H:%M') + f'\nCPF: {self.cpf} and Name: {self.name}\n\n'
        
#         idServidor, remuneracaoBasicaBruta = remuneracao[['Id_SERVIDOR_PORTAL', 'REMUNERAÇÃO BÁSICA BRUTA (R$)']].values[0]

In [ ]:
class Extract_Image_Text:
    def __init__(self, path, credentials):
        self.clientVision = vision.ImageAnnotatorClient(credentials=credentials)

        self.content = io.open(path, 'rb').read()

        self.image = vision.Image(content=self.content)
        self.cv2image = cv2.imread(path)
    
    def text_detection(self):
        return self.clientVision.text_detection(image=self.image)

    def document_text_detection(self):
        return self.clientVision.document_text_detection(image=self.image)
    

def get_text_annotations(visionObj):
    objDescription = visionObj.text_annotations[0].description
    return {'splited':objDescription.split('\n'), 'concatenated':objDescription.replace('\n', '')}


In [11]:
a = r'/mnt/e/Aram/Projeto_KGB/Documentos/015.639.431-66/015.639.431-66_CPF_345230_19042021144053048.JPG'
b = r'/mnt/e/Aram/Projeto_KGB/Documentos/015.639.431-66/015.639.431-66_DOC IDENT V_345230_19042021144053045.JPG'

im1 = cv2.imread(a)
im2 = cv2.imread(b)


def MSE(imageA, imageB):
	err = np.sum(np.square(imageA.astype("float") - imageB.astype("float")))
	err /= float(imageA.shape[0] * imageA.shape[1])
	return err


assert im1.shape == im2.shape and MSE(im1, im2) <= 200, f'SAME TYPE OF DOCUMENT (IMAGE) DIFERENT CONTENT, MSE: {im1} <> {im2}\n'

In [20]:
# Como verificar holerite
# O que eu ja fiz
# verificar assinatura Sim

pathKGB = path.normpath(r'/mnt/e/Aram/Projeto_KGB')
pathDir = path.join(pathKGB, 'Documentos')
foldersCPFsNames = listdir(pathDir)



class Client:
    def __init__(self, cpf, pathFichaCadastral):
        self.cpf = cpf
        self.pathFichaCadastral = pathFichaCadastral
        
        dictObjsDataFichaCadastral = self.process_ficha_cadastral()
        
        self.name = dictObjsDataFichaCadastral['nome completo:'].get_text().replace('\n', '').lower()
        self.dataNascimento = datetime.strptime(dictObjsDataFichaCadastral['data nascimento:'].get_text().replace('\n', ''), "%d/%m/%Y")
        self.idade = round((today - self.dataNascimento).days/365.25, 3)
        
        # Check CPF
        assert self.cpf == dictObjsDataFichaCadastral['cpf:'].get_text().replace('\n', ''), today.strftime('%d/%m/%Y, %H:%M') + f' <Client>\n{self.cnpj} NOT FOUND IN "FICHA CADASTRAL"'
        
        self.nDocIdentidade = dictObjsDataFichaCadastral['nº documento de identidade:'].get_text().replace('\n', '').lower()
        self.uf = dictObjsDataFichaCadastral['uf:'].get_text().replace('\n', '').lower()
        self.cep = dictObjsDataFichaCadastral['cep:'].get_text().replace('\n', '')
        
        self.dataContratacao = datetime.strptime(dictObjsDataFichaCadastral['tempo de serviço:'].get_text().replace('\n', ''), "%d/%m/%Y")
        self.ondeTrabalha = dictObjsDataFichaCadastral['empresa/orgão:'].get_text().replace('\n', '').lower()
        self.salario = float(dictObjsDataFichaCadastral['salário:'].get_text().replace('\n', '').replace('R$', '').replace(',', '-').replace('.', '').replace('-', '.'))


    def parse_obj(self, lt_objs):
        # Processa os objetos do pdf
        def distance_boxes_RL_DT(box1, box2):
            #  _____
            # |____|
            #      \_____
            #      |____|
            return (box1[2] - box2[0])**2 + (box1[1] - box2[1])**2
        
        listCampos = ['nome completo:', 'cpf:', 'data nascimento:', 'cep:', 'empresa/orgão:',
                      'uf:', 'nº documento de identidade:', 'tempo de serviço:', 'salário:']
        dictCampos = {}
        state = True
        
        for obj in lt_objs:
            if isinstance(obj, LTTextBoxHorizontal):
                if state:
                    valToEverybody = obj
                    state = False
                
                for campo in listCampos:
                    if obj.get_text().replace('\n', '').lower() == campo:
                        try:
                            if obj.bbox[1] > dictCampos[campo].bbox[1]:
                                dictCampos[campo] = obj
                        except:
                            dictCampos[campo] = obj
                        break

        dictCamposVals = {_:valToEverybody for _ in listCampos}

        for obj in lt_objs:
            if isinstance(obj, LTTextBoxHorizontal):
                
                for campo in listCampos:
                    if (
                        dictCampos[campo].bbox[0] < obj.bbox[0] and obj.bbox[1] <= dictCampos[campo].bbox[3] and
                        distance_boxes_RL_DT(dictCampos[campo].bbox, obj.bbox) < distance_boxes_RL_DT(dictCampos[campo].bbox, dictCamposVals[campo].bbox)
                    ): dictCamposVals[campo] = obj
        
        return dictCamposVals


    def process_ficha_cadastral(self):
        # Lê o pdf "FICHA CADASTRAL"
        with open(self.pathFichaCadastral, 'rb') as fichaCadastral:
            parser = PDFParser(fichaCadastral)
            doc = PDFDocument(parser)
            rsrcmgr = PDFResourceManager()
            device = PDFDevice(rsrcmgr)
            device = PDFPageAggregator(rsrcmgr, laparams=LAParams())
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            
            for page in PDFPage.create_pages(doc):
                interpreter.process_page(page)
                layout = device.get_result()

                dictCamposVals = self.parse_obj(layout._objs)
            fichaCadastral.close()

        return dictCamposVals


class CCB:
    def __init__(self, client, pathCCB):
        self.client = client
        self.pathCCB = pathCCB
        
        self.clienteCCB, self.CCB = self.process_CCB()
    

    def parse_obj(self, lt_objs):
        # Processa os objetos do pdf
        def distance_boxes_RL_TT(box1, box2):
            #  ______________
            # |____|   |____|
            return (box1[2] - box2[0])**2 + (box1[3] - box2[3])**2
        
        
        def distance_boxes_centerL_LR(box1, box2):
            #   _____
            #  |____|
            #  ___|_
            # |____|
            return ((box1[0] + box1[2]*2)/3 - (box2[0] + box2[2])/2)**2 + (box1[1] - box2[3])**2
            
        listCamposInfosCliente = ['nome/razão social:', 'cpf/cnpj:', 'cep:', 'estado:', 'rg:',
                                  '5.1. valor total da dívida:']# TODO: 'assinatura emitente:'
        listCamposInfosCCB = ['céduladecréditobancário', '5.vencimento-tododia', 'quantidadedeparcelas',
                              'últimovencimento', 'primeirovencimento', 'totaldaparcela', 'principal', 'juros']
        
        dictCamposInfosCliente = {}
        dictCamposInfosCCB = {}
                
        for obj in lt_objs:
            if isinstance(obj, LTTextBoxHorizontal):
                if len(dictCamposInfosCliente) == 2:
                    valToEverybody = obj
                
                for campo in listCamposInfosCliente + listCamposInfosCCB:
                    objTxtReplacedLower = obj.get_text().replace('\n', '').replace(' ', '').lower()
                    
                    if re.search('^' + campo, objTxtReplacedLower) != None and campo in listCamposInfosCCB:
                        try:
                            if obj.bbox[1] > dictCamposInfosCCB[campo].bbox[1]:
                                dictCamposInfosCCB[campo] = obj
                        except:
                            dictCamposInfosCCB[campo] = obj
                        break
                    
                    elif obj.get_text().replace('\n', '').lower() == campo:
                        try:
                            if obj.bbox[1] > dictCampos[campo].bbox[1]:
                                dictCamposInfosCliente[campo] = obj
                        except:
                            dictCamposInfosCliente[campo] = obj

            # Pegas as linhas (que formam tabelas) da pagina
#             elif isinstance(obj, LTLine):
#                 pass
        
        dictCamposInfosClienteVals = {_:valToEverybody for _ in listCamposInfosCliente}
        dictCamposInfosCCBVals = {_:valToEverybody for _ in listCamposInfosCCB}

        for obj in lt_objs:            
            if isinstance(obj, LTTextBoxHorizontal):
                for campo in listCamposInfosCliente:
                    if (
                        dictCamposInfosCliente[campo].bbox[2] < obj.bbox[0] and
                        distance_boxes_RL_TT(dictCamposInfosCliente[campo].bbox, obj.bbox) < distance_boxes_RL_TT(dictCamposInfosCliente[campo].bbox, dictCamposInfosClienteVals[campo].bbox)
                    ): dictCamposInfosClienteVals[campo] = obj
                
                for campo in listCamposInfosCCB:
                    if (
                        obj != dictCamposInfosCCB[campo] and obj.bbox[1] < dictCamposInfosCCB[campo].bbox[1] and # obj.bbox[3] < dictCamposInfosCCB[campo].bbox[3] and
                        distance_boxes_centerL_LR(dictCamposInfosCCB[campo].bbox, obj.bbox) < distance_boxes_centerL_LR(dictCamposInfosCCB[campo].bbox, dictCamposInfosCCBVals[campo].bbox)
                    ): dictCamposInfosCCBVals[campo] = obj
        
        return dictCamposInfosClienteVals, dictCamposInfosCCBVals
            
    def process_CCB(self):
        # Lê o pdf "CCB"
        with open(self.pathCCB, 'rb') as ccb:
            parser = PDFParser(ccb)
            doc = PDFDocument(parser)
            rsrcmgr = PDFResourceManager()
            device = PDFPageAggregator(rsrcmgr, laparams=LAParams(line_margin=0.35))
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            
            for page in PDFPage.create_pages(doc):
                interpreter.process_page(page)
                layout = device.get_result()

                dictCliente, dictCCB = self.parse_obj(layout._objs)
                break
            ccb.close()
        
        return dictCliente, dictCCB


class Checker:
    def __init__(self, cpf, pathDocuments):
        self.cpf = cpf
        self.pathDocuments = pathDocuments
        
        filesNames = listdir(self.pathDocuments)

        self.paths = {_: list() for _ in ['fichaCadastral', 'gravacao', 'ccb', 'holerite', 'comp', 'averbacao']}
        self.paths.update({'docs':{}, 'others':{}})
        
        fileNotCoresponded = []
        
        # Validar CPF
        CPFValidator = CPF()
        assert CPFValidator.validate(self.cpf), 'NOT VALID CPF ' + today.strftime('%d/%m/%Y, %H:%M') + f' - cpf: {self.cpf}\n\n'

        for cont, fileName in enumerate(filesNames):
            listNameParts = fileName.split('_')
            pathFile = path.join(self.pathDocuments, fileName)
            
            if listNameParts[0] == folderDocumentsName:
                if listNameParts[1].lower().replace(' ', '')[:14] == 'fichacadastral':
                    self.paths['fichaCadastral'].append(pathFile)
                elif listNameParts[1].lower() == 'gravacao':
                    self.paths['gravacao'].append(pathFile)
                elif listNameParts[1].lower() == 'holerite':
                    self.paths['holerite'].append(pathFile)
                elif listNameParts[1].lower()[:3] == 'ccb':
                    self.paths['ccb'].append(pathFile)
                elif listNameParts[1].lower() == 'cpf':
                    try:
                        self.paths['docs']['cpf'].append(pathFile)
                    except:
                        self.paths['docs']['cpf'] = []
                        self.paths['docs']['cpf'].append(pathFile)
                elif listNameParts[1].lower().replace(' ', '')[:8] == 'docident':
                    try:
                        self.paths['docs']['docs'].append(pathFile)
                    except:
                        self.paths['docs']['docs'] = []
                        self.paths['docs']['docs'].append(pathFile)
                elif listNameParts[1].lower() == 'comp':
                    self.paths['comp'].append(pathFile)
                elif listNameParts[1].lower() == 'averbacao':
                    self.paths['averbacao'].append(pathFile)
                else:
                    self.paths['others'][listNameParts[1].lower()] = pathFile
            else:
                # Colocar no arquivo de erros que foi encontrado um arquivo sem
                # o cpf no começo levanta um erro que não comprometa o codigo
                fileNotCoresponded.append(pathFile)
        
        # TODO: Verificar Duplicatas
        
        # Passa os outros para um aruivo, para fins de controle
        if self.paths['others'] or fileNotCoresponded:
            pathOthersFile = path.join(pathKGB, 'Others.txt')
            with open(pathOthersFile, 'a') as othersFile:
                othersFile.write(today.strftime('%d/%m/%Y, %H:%M') + '\n' + dumps({self.cpf:{'Others':list(self.paths['others'].items()), 'Not coresponded':fileNotCoresponded}}, indent=4) + '\n\n\n')
                othersFile.close()
        del fileNotCoresponded
        
        # Da um AssertionError caso algum dos tipos de documento obrigatorio
        # não tiver nenhum arquivo relacionado
        errores = []
        for fileType in self.paths.keys():
            if fileType == 'docs' and not self.paths[fileType]['cpf']: # Checa se não existe um documento CPF
                errores.append('cpf')
            elif not self.paths[fileType] and fileType != 'others': # Checa se não algum dos outros não existe menos 'others'
                errores.append(fileType)
        assert not errores, 'FILE NOT FOUND ' + today.strftime('%d/%m/%Y, %H:%M') + '\n' + dumps({'cpf':self.cpf, 'Files Types':[_ for _ in errores]}, indent=4) + '\n\n\n'
        del errores

        self.cliente = Client(self.cpf, self.paths['fichaCadastral'][0])
        
        self.CCB = CCB(self.cliente, self.paths['ccb'][0])
        
        objReceitaHolerite, *_ = self.validate_holerite()
        receitaHolerite = float(objReceitaHolerite.get_text().replace('\n', '').replace('R$', '').replace(',', ' ').replace('.', '').replace(' ', '.'))
        assert self.cliente.salario == receitaHolerite, '"Receita/Salário" NOT CORRESPONDENT ' + today.strftime('%d/%m/%Y, %H:%M') + f'\ncpf: {self.cpf}, name: {self.cliente.name},' +\
                                                        f'\nReseita holerite: {str(receitaHolerite)}, salario ficha: {str(self.cliente.salario)}'
#         self.validate_antifraude()
        
        self.validate_transparencia()
        
        # TODO: holerite bater com portal maximo 10% de diferença para menos
    
    def validate_holerite(self):
        def parse_obj(lt_objs):
            def distance_boxes_centerL_LR(box1, box2):
                #   _____
                #  |____|
                #  __|__
                # |____|
                return ((box1[0] + box1[2])/2 - (box2[0] + box2[2])/2)**2 + (box1[1] - box2[3])**2
            
            listVariaveisCampoReceita = ['bruto', 'receita']
            listDadosCliente = [self.cpf, self.cliente.name]
            
            cpfAsNumber = self.cpf.replace('.', '').replace('-', '')
            names = self.cliente.name.split(' ')
            names = [names[0], names[-1]]
            
            for obj in lt_objs:
                if isinstance(obj, LTTextBoxHorizontal):
                    objTextLower = obj.get_text().replace('\n', '').lower()

                    if objTextLower.find(listVariaveisCampoReceita[0]) != -1 or objTextLower.find(listVariaveisCampoReceita[1]) != -1:
                        try:
                            if obj.bbox[1] < objReceita.bbox[1]:
                                objReceita = obj
                        except:
                            objReceita = obj
                                
                    if objTextLower.replace('.', '').replace('-', '').find(cpfAsNumber) != -1:
                        objCpf = obj
                    else:
                        if objTextLower.find(names[0]) != -1 and objTextLower.find(names[1]) != -1:
                            objName = obj
            
            objReceitaValor = objReceita
            
            for obj in lt_objs:
                if isinstance(obj, LTTextBoxHorizontal):
                    if (
                        obj.bbox != objReceita.bbox and
                        distance_boxes_centerL_LR(objReceita.bbox, obj.bbox) < distance_boxes_centerL_LR(objReceita.bbox, objReceitaValor.bbox)
                    ): objReceitaValor = obj
            
            assert 'objName' in locals() and 'objCpf' in locals(), 'CPF OR NAME NOT FOUND IN HOLERITE ' + today.strftime('%d/%m/%Y, %H:%M') + f'\ncpf: {self.cpf}, name: {self.cliente.name}'
            
            return (objReceitaValor, objName, objCpf)
                    
        
        with open(self.paths['holerite'][0], 'rb') as holerite:
            parser = PDFParser(holerite)
            doc = PDFDocument(parser)
            rsrcmgr = PDFResourceManager()
            device = PDFPageAggregator(rsrcmgr, laparams=LAParams(line_margin=0.2))
            interpreter = PDFPageInterpreter(rsrcmgr, device)

            for page in PDFPage.create_pages(doc):
                interpreter.process_page(page)
                layout = device.get_result()

                tupObjs = parse_obj(layout._objs)
                break
                
            holerite.close()
        
        return tupObjs
            
    
    def validate_antifraude(self):
        def parse_obj(lt_objs):
            def distance_boxes_center_LR(box1, box2):
                #  _____     _____
                # |____|----|____|
                return (box1[2] - box2[0])**2 + ((box1[1] + box1[3])/2 - (box2[1] + box2[3])/2)**2

            
            def distance_boxes_left_TD(box1, box2):
                #  _____
                # |____|
                # \_____
                # |____|
                return (box1[0] - box2[0])**2 + (box1[1] - box2[3])**2
            
            
            listCamposCliente, listCamposStatus = ['nome', 'cpf'], ['status:', 'motivo:', 'observação:']

            dictCampos = {}

            for obj in lt_objs:
                if isinstance(obj, LTTextBoxHorizontal):
                    if len(dictCampos) == 2 and not obj.get_text().replace('\n', '').lower() in listCamposCliente + listCamposStatus:
                        valToEverybody = obj

                    for campo in listCamposCliente + listCamposStatus:

                        if obj.get_text().replace('\n', '').lower() == campo or obj.get_text().replace('\n', '').lower().split(' ')[0] == campo:
                            try:
                                if obj.bbox[1] > dictCampos[campo].bbox[1]:
                                    dictCampos[campo] = obj
                            except:
                                dictCampos[campo] = obj
            
            dictCamposVals = {_:valToEverybody for _ in listCamposCliente + listCamposStatus}            

            for obj in lt_objs:
                if isinstance(obj, LTTextBoxHorizontal):
                    for campo in listCamposCliente:
                        if (
                            obj.bbox[3] < dictCampos[campo].bbox[1] and
                            distance_boxes_left_TD(dictCampos[campo].bbox, obj.bbox) < distance_boxes_left_TD(dictCampos[campo].bbox, dictCamposVals[campo].bbox)
                        ): dictCamposVals[campo] = obj

                    for campo in listCamposStatus:
                        if (
                            dictCampos[campo].bbox[2] < obj.bbox[0] and
                            distance_boxes_center_LR(dictCampos[campo].bbox, obj.bbox) < distance_boxes_center_LR(dictCampos[campo].bbox, dictCamposVals[campo].bbox)
                        ): dictCamposVals[campo] = obj
            
            
        with open(self.paths['others']['antifraude'], 'rb') as antifraude:
            parser = PDFParser(antifraude)
            doc = PDFDocument(parser)
            rsrcmgr = PDFResourceManager()
            device = PDFPageAggregator(rsrcmgr, laparams=LAParams())
            interpreter = PDFPageInterpreter(rsrcmgr, device)

            for page in PDFPage.create_pages(doc):
                interpreter.process_page(page)
                layout = device.get_result()

                dictAntifraude = parse_obj(layout._objs)
                break

            antifraude.close()
    
    
    def validate_transparencia(self):
        head = {'accept': '*/*', 'chave-api-dados': 'cb76e5b1a35c4a24e84b7fde2d1a0244'}

        cpf = self.cpf.replace('.', '').replace('-', '')
        requestTransparencia = requests.get(f'http://api.portaldatransparencia.gov.br/api-de-dados/servidores/remuneracao?cpf={cpf}&mesAno={str(202009)}&pagina=1', headers=head)

        assert requestTransparencia.status_code != 404, '<404> NOT FOUND IN "portaldatransparencia.gov.br"' + today.strftime('%d/%m/%Y, %H:%M') + f'\ncpf: {self.cpf}, name: {self.cliente.name}'
        assert requestTransparencia.status_code != 403, '<403> FORBIDDEN "portaldatransparencia.gov.br"' + today.strftime('%d/%m/%Y, %H:%M') + f'\ncpf: {self.cpf}, name: {self.cliente.name}'
        assert requestTransparencia.status_code != 401, '<401> UNAUTHORIZED "portaldatransparencia.gov.br"' + today.strftime('%d/%m/%Y, %H:%M') + f'\ncpf: {self.cpf}, name: {self.cliente.name}'
        
#         print(json.loads(requestTransparencia.text)[0]['remuneracoesDTO'])#['remuneracoesDTO'])
        pessoa = json.loads(requestTransparencia.text)[0]['servidor']['pessoa']
        name = pessoa['nome'].strip().lower()

        assert self.cliente.name == name, 'NAME NOT CORRESPONDENT IN "portaldatransparencia.gov.br"' + today.strftime('%d/%m/%Y, %H:%M') + f'\ncpf: {self.cpf}, name: {self.cliente.name}'


    def check_duplicates_PDFs(self):
        c = r'/mnt/e/Aram/Projeto_KGB/Documentos/929.644.434-91/929.644.434-91_HOLERITE_344856_19042021144052230.PDF'
        d = r'/mnt/e/Aram/Projeto_KGB/Documentos/929.644.434-91/929.644.434-91_HOLERITE_344856_19042021144052284.PDF'

        paths = [c, d]

        for cont, pathPDF in enumerate(paths):
            with open(pathPDF, 'rb') as in_file:
                parser = PDFParser(in_file)
                doc = PDFDocument(parser)
                rsrcmgr = PDFResourceManager()
                device = TextConverter(rsrcmgr, output_strings[cont], laparams=LAParams())
                interpreter = PDFPageInterpreter(rsrcmgr, device)
                for page in PDFPage.create_pages(doc):
                    interpreter.process_page(page)


        assert output_strings[0].getvalue() == output_strings[1].getvalue(), 'SAME TYPE OF DOCUMENT (PDF) DIFERENT CONTENT ' + today.strftime('%d/%m/%Y, %H:%M') + f'\n{self.cpf}; Doc Type: {key}'

    def check_duplicates_images(self):
        im1 = cv2.imread(a)
        im2 = cv2.imread(b)


        def MSE(imageA, imageB):
            err = np.sum(np.square(imageA.astype("float") - imageB.astype("float")))
            err /= float(imageA.shape[0] * imageA.shape[1])
            return err


        assert im1.shape == im2.shape and MSE(im1, im2) <= 200, f'SAME TYPE OF DOCUMENT (IMAGE) DIFERENT CONTENT, MSE: {im1} <> {im2}\n'


today = datetime.now()

folderDocumentsName = foldersCPFsNames[1]
pathDocuments = path.join(pathDir, folderDocumentsName)

Checker(folderDocumentsName, pathDocuments)



In [29]:
pathPDF = r'/mnt/e/Aram/Projeto_KGB/Documentos/015.639.431-66/015.639.431-66_HOLERITE_345230_19042021144052970.PDF'

output_string = StringIO()
with open(pathPDF, 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    print(doc.info)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)


from nltk.corpus import treebank

from nltk.corpus import stopwords
from string import punctuation

stopwords = set(stopwords.words('portuguese') + list(punctuation))

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

sentencas = sent_tokenize(output_string.getvalue().replace('\n', ' '), language='portuguese')
palavras = word_tokenize(output_string.getvalue().replace('\n', ' ').lower(), language='portuguese')

palavrasNoStopW = [palavra for palavra in palavras if palavra not in stopwords]

t = nltk.pos_tag(['14/07/21'])
t

* #### Problema data
    * Como não estou conseguindo pegar a data posso: 
        * Pegar a data de criação nos metadados
        * Se ela for anterior a 3 meses (90 dias) ja discartamos se não seguimos
        * E bater os dados com os do portal da transparencia
        * Caso forem diferentes dos dados dos ultimos 3 meses do portal damos um erro
    * Pros:
        * Robusto (OBS)
        * Eficiente
        * Vai funcionar (na maioria dos casos)
    * Contras:
        * Não ira funcionar com militares
        * Não sera muito robusto na questão de pegar o salario liquído (não tem padrao)
    * Outra opção
        * Fazer um programa que teste todas as opções possiveis para uma data (ex: 'jan21', '1/21', janeiro 2021')
        * OBS:
            * De qualquer forma não teremos certeza que aquela data se refere a data do salario do mes x/de competencia pois em todos os documentos os campos tem nomes diferentes
            * So seria possivel fazer isso apos termos uma boa amostra com todos os nomes possiveis para os campos
                * E mesmo assim o cogigo seria extremamente ineficiente


In [109]:
# import parsedatetime
# cal = parsedatetime.Calendar()

# cal.parse('dez 2021')
# to = {'a':'b'}
# 'acac a c'.translate(to)



90